# Sioux Falls

Parameter perturbation analysis with the Sioux Falls instance. The graph was took from the TransportationNetworks git repository and the demand matrix from the Liu paper and consists of 22 pairs origin-destination.

The parameters that are analyzed are:
- Budget
- Breakpoints
- Infrastructure count

In [1]:
import os
from pprint import pp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import bcnetwork as bc

## Reading data

The models and solution data is read from a directory populated elsewhere.

In [19]:
def get_solution_path(model_path):
    dirname, basename = os.path.split(model_path)
    name, _ = os.path.splitext(basename)
    
    return os.path.join(dirname, f'solution_{name}.pkl')


def get_row_from_model(model_path):
    """
    Return param information and run information.
    """
    solution_path = get_solution_path(model_path)
    
    if not os.path.exists(solution_path):
        return None

    model = bc.model.Model.load(model_path)
    solution = bc.solution.Solution.load(solution_path)

    return {
        'infrastructure_count': model.infrastructure_count,
        'budget': model.budget,
        'budget_factor': model._budget_factor,
        'breakpoint_count': len(model.breakpoints),
        'total_demand_transfered': solution.total_demand_transfered,
        'budget_used': solution.budget_used,
        'run_time_seconds': solution.run_time_seconds,
    }

        
def generate_runs_dataframe(working_dir):
    """
    Return a pd.DataFrame out of a directory of
    models and solutions.
    """
    rows = []
    
    for entry in os.scandir(working_dir):
        if entry.is_dir() or not entry.name.endswith('.pkl') or 'solution' in entry.name:
            continue
        
        data = get_row_from_model(os.path.join(working_dir, entry.name))
        if data:
            rows.append(data)

    return pd.DataFrame(rows)        

In [20]:
data_dir = '../sensitivity'
dataframe_cache = os.path.join(data_dir, 'runs.csv')

if False and os.path.exists(dataframe_cache):
    df = pd.read_csv(dataframe_cache)
else:
    df = generate_runs_dataframe(data_dir)
    df.to_csv()

In [22]:
df[df.total_demand_transfered == 0]

,infrastructure_count,budget,budget_factor,breakpoint_count,total_demand_transfered,budget_used,run_time_seconds
3,2,42.142105,0.134211,2,0,42,2.488183
5,4,3.140000,0.010000,2,0,3,1.270658
22,2,61.643158,0.196316,2,0,61,1.318459
42,2,81.144211,0.258421,2,0,81,1.540008
77,2,129.896842,0.413684,2,0,129,1.561716
79,2,12.890526,0.041053,2,0,12,1.435853
88,2,32.391579,0.103158,2,0,32,2.405204
93,2,71.393684,0.227368,2,0,71,1.575376
95,2,139.647368,0.444737,2,0,139,1.356941
103,2,188.400000,0.600000,2,0,186,0.723786


In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()

ax.set_ylabel('Total demand transfered')
ax.set_xlabel('Budget Factor')
ax2.set_ylabel('Run Time')


ax.plot(budget_factors[:7], [s.total_demand_transfered for s in budget_factors_solutions], color=bc.colors.green)
ax2.plot(budget_factors[:7], [s.run_time_seconds for s in budget_factors_solutions], color=bc.colors.orange)